In [ ]:
import laspy
import numpy as np
from scipy.interpolate import griddata
import rasterio
import pyproj
import cv2


# Enter your .las/.laz data name here
las_file = laspy.read('Yourdata.laz')
# Set the CRS information to projection
las_file.header.scale[0] = 0.01
las_file.header.scale[1] = 0.01
las_file.header.scale[2] = 0.01
las_file.write("updated_file.las")
las_file = laspy.read('updated_file.las')



# Get the input EPSG from user
input_epsg = int(input("Enter the input EPSG code: "))

# Create a Pyproj CRS object for the input EPSG code
input_crs = pyproj.CRS.from_epsg(input_epsg)


# Extract the x, y, and z coordinates from the LAS file
x = las_file.x
y = las_file.y
z = las_file.z

# Define the resolution of the dsm
resolution = 1

# Calculate the grid bounds based on the x and y coordinates
x_min = np.floor(min(x))
x_max = np.ceil(max(x))
y_min = np.floor(min(y))
y_max = np.ceil(max(y))

# Generate the grid of points for the dsm
grid_x, grid_y = np.meshgrid(np.arange(x_min, x_max , resolution), 
                             np.arange(y_min, y_max , resolution))

# Generate the dsm using Nearest Neighbour interpolation of the point cloud
dsm = griddata((x, y), z, (grid_x, grid_y), method='nearest')


# Save the dsm to a GeoTIFF file using rasterio
with rasterio.open(
        'dsm.tif',
        'w',
        driver='GTiff',
        height=dsm.shape[0],
        width=dsm.shape[1],
        count=1,
        dtype=dsm.dtype,
        crs=input_crs,
        transform=rasterio.transform.Affine(resolution, 0, x_min, 0, resolution, y_min)) as dst:
    dst.write(dsm, 1)
print('Success')

In [3]:
from rasterio.transform import from_origin
from scipy.interpolate import griddata
import scipy


# Load LiDAR data
las_file_dtm = laspy.read('updated_file.las')
points = np.vstack((las_file_dtm.x, las_file_dtm.y, las_file_dtm.z)).T

# Classify ground points in lidar data class 2
ground_points = points[las_file_dtm.classification == 2]

# Define the resolution of the output DTM
resolution = 1

# Determine the bounds of the point cloud
min_x, max_x = np.min(points[:,0]), np.max(points[:,0])
min_y, max_y = np.min(points[:,1]), np.max(points[:,1])

# Calculate the size of the output raster
width = int(np.ceil((max_x - min_x) / resolution))
height = int(np.ceil((max_y - min_y) / resolution))

# Create the output raster profile
profile = {
    'driver': 'GTiff',
    'height': height,
    'width': width,
    'count': 1,
    'dtype': 'float32',
    'crs': input_crs,
    'transform': from_origin(min_x, min_y, resolution, -resolution)
}

# Create an empty numpy array for the output DTM
dtm = np.zeros((height, width), dtype=np.float32)
counts = np.zeros((height, width), dtype=np.int32)

# Create a KDTree from the x, y coordinates of the ground points
tree = scipy.spatial.cKDTree(ground_points[:, :2])

# Create a mesh grid for the output raster
mesh_x, mesh_y = np.meshgrid(np.arange(min_x, max_x, resolution), np.arange(min_y, max_y, resolution))

# Interpolate the z values of the ground points onto the mesh grid
values = tree.query(np.vstack((mesh_x.ravel(), mesh_y.ravel())).T)[0]
mesh_z = values.reshape(mesh_x.shape)


# Load the DSM
dsm_file = laspy.read('updated_file.las')
dsm_points = np.vstack((dsm_file.x, dsm_file.y, dsm_file.z)).T

# Classify points as aboveground features (non-ground points)
non_ground_points = dsm_points[dsm_file.classification != 2]

# Interpolate the non-ground points onto the mesh grid
non_ground_z = griddata(non_ground_points[:, :2], non_ground_points[:, 2], (mesh_x, mesh_y), method='nearest')

# Subtract the interpolated non-ground values from the interpolated ground values
dtm = 10*mesh_z - non_ground_z

# Write the output raster to a file
with rasterio.open('dtm.tif', 'w', **profile) as dst:
    dst.write(dtm, 1)

In [4]:
import rasterio
from rasterio.warp import calculate_default_transform, reproject, Resampling


# Load DSM and DTM
with rasterio.open('dsm.tif') as src:
    dsm = src.read(1)
    dsm_meta = src.profile

with rasterio.open('dtm.tif') as src:
    dtm = src.read(1)

# Compute NDHM
ndhm = dsm - dtm

# Write NDHM to file
ndhm_meta = dsm_meta.copy()
ndhm_meta['dtype'] = 'float32'
with rasterio.open('ndhmtemp.tif', 'w', **ndhm_meta) as dst:
    dst.write(ndhm.astype(np.float32), 1)
    
# Define the target CRS as EPSG:3857
target_crs = 'EPSG:3857'

# Open the input file
with rasterio.open('ndhmtemp.tif') as src:
    # Get the metadata of the input file
    src_profile = src.profile.copy()

    # Calculate the transform to the target CRS
    dst_transform, dst_width, dst_height = calculate_default_transform(
        src.crs, target_crs, src.width, src.height, *src.bounds)

    # Update the metadata of the output file with the target CRS
    src_profile.update({
        'crs': target_crs,
        'transform': dst_transform,
        'width': dst_width,
        'height': dst_height})

    # Create the output file
    with rasterio.open('ndhm.tif', 'w', **src_profile) as dst:
        # Reproject the input file to the target CRS
        reproject(
            source=rasterio.band(src, 1),
            destination=rasterio.band(dst, 1),
            src_transform=src.transform,
            src_crs=input_crs,
            dst_transform=dst_transform,
            dst_crs=target_crs,
            resampling=Resampling.nearest)

In [ ]:
# You can change the last 2 parameters of cv2.adaptiveThreshold for morphological kernel size, squareness,
# contour size, and width parameters to determine the strictness of the detection algorithm.

from rasterio.plot import show
from rasterio.mask import mask
import cv2
import numpy as np

# Open the NDHM image
with rasterio.open('ndhm.tif') as src:
    ndhm_img = src.read(1)
    profile = src.profile.copy()

# Threshold the NDHM image to separate the buildings
# Convert the NDHM image to 8-bit
ndhm_img_8bit = cv2.normalize(ndhm_img, None, 0, 255, cv2.NORM_MINMAX, cv2.CV_8UC1)

# Apply adaptive thresholding to separate the buildings
ndhm_img_thresh = cv2.adaptiveThreshold(ndhm_img_8bit, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY_INV, 51, 4.6)

# Apply morphological opening to remove small objects and fill gaps
kernel = np.ones((3, 3), np.uint8)
ndhm_img_open = cv2.morphologyEx(ndhm_img_thresh, cv2.MORPH_OPEN, kernel)

# Find the contours of the buildings
contours, _ = cv2.findContours(ndhm_img_open.astype(np.uint8), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

# Create a binary mask for the buildings
building_mask = np.zeros_like(ndhm_img_open, dtype=np.uint8)

# Set a minimum and maximum building size
min_size = 35
max_size = 5000

# Get the pixel size in feet instead of meters
pixel_size = abs(profile['transform'][0])

# Filter out contours with low squareness or small size (likely vegetation or noise)
for contour in contours:
    # Compute the squareness and size of the contour
    rect = cv2.minAreaRect(contour)
    w, h = rect[1]
    if w < h:
        w, h = h, w
    squareness = w / h if h != 0 else 0
    size = w * h * pixel_size ** 2

    # Filter out contours with low squareness or small size (likely vegetation or noise)
    if squareness >= 0.3 and min_size <= size <= max_size and w >= 3 and h >= 3:
        cv2.drawContours(building_mask, [contour], -1, 255, -1)

# Apply morphological closing to fill remaining small holes in the buildings
kernel = np.ones((15, 15), np.uint8)
building_mask_closed = cv2.morphologyEx(building_mask, cv2.MORPH_CLOSE, kernel)

# Update the metadata of the output file to reflect a binary output
profile.update(count=1, dtype=rasterio.uint8)

# Write the new GeoTIFF file with the binary mask of the buildings
with rasterio.open('buildings.tif', 'w', **profile) as dst:
    dst.write(building_mask_closed.astype(rasterio.uint8), 1)

print('All of our steps are done.')

